In [2]:
import os
import numpy as np
import pydicom

import pandas as pd
import sys
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/read_weasis_raw_v0.96/')

import weasis_raw_data_api as wr
sys.path.append('/mnt/fast-disk1/mjc/utils_codes/')
from utils_test import *
from utils_metrics_3d import *

D_dir2header_df = {}
def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:

                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        
                        if metadata in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                            print( 'error of loading key: {}'.format(metadata) )                    
                        else:
                            data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [3]:
def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        try:
            df[col] = df[col].str.replace(ori,new, case = False) 
        except:
            pass
            
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instance should be str or list')


def str_Xdrive2mnt(df_all):
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/Y-drive")
    pd_str_replace(df_all, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
    pd_str_replace(df_all, ['Image File Path'], "/mnt/Y-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")

    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen",
    "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_Modified_By_Yen")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    pd_str_replace(df_all, ['Contour File Path'], "/mnt/Y-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
    
def get_onect_from_list(df_list , ct):
    for i, df in enumerate(df_list):
        try:
            df_ct = df[ (df["Image File Path"]==ct) & (df['Location'].isin(['liver'])) ]
        except KeyError:
            df_ct = df[ (df["Image File Path"]==ct)]
        if df_ct.shape[0]:
            return df_ct , i
    print("warning! no CT was found")
    return None ,None

def raws2mask(raws , D_z_index, mask_vol = None):

    for raw in raws:

        radiologist_raw = wr.read(raw)
        slice_list = radiologist_raw.get_instance_number_array()
        if mask_vol is None:
            mask_vol = initialize_mask_vol(radiologist_raw , D_z_index)
        for j, one in enumerate(slice_list):
            mask = radiologist_raw.get_mask_image(j)
            mask_vol[D_z_index[one]] += mask
    return mask_vol

In [4]:
folder = '/mnt/fast-data/mjc/AutoRECIST/Inputs/'

df_CTs = pd.read_excel(folder+'AutoRECIST_List_LesionSize_20220602_JM_SingleCTSeries.xlsx')
str_Xdrive2mnt(df_CTs)

df_CTs = df_CTs[df_CTs['dataset']=='CUIMCfirst']
print(df_CTs)

                                              Comments  Patient ID  \
153       minor revision(falsely targeted fat deposit)  METNET0002   
154               minor revision(falsely targeted IVC)  METNET0003   
155                                     minor revision  METNET0004   
156                                     minor revision  METNET0006   
157                                     minor revision  METNET0008   
..                                                 ...         ...   
220                             minimal or no revision  METNET0268   
221                             minimal or no revision  METNET0277   
222                             minimal or no revision  METNET0289   
223       minor revision(falsely targeted gallbladder)  METNET0294   
224  minor revision(falsely targeted dilated biliar...  METNET0296   

                                       Image File Path  \
153  /mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...   
154  /mnt/fast-disk1/mjc/AutoRecist/Inputs/

In [5]:
df_Yen = pd.read_excel(folder+'PDS_AUTO_RECIST CIA-LAB Testing Dataset Gold Standard_Yen_2022-06-13.xlsx')
str_Xdrive2mnt(df_Yen)

AI_raw_list = ['ScaleNAS9SlicesAccLiverToRaw_Test353.csv',]
df_AIs = []
for one in AI_raw_list:
    df = pd.read_csv(one)
    str_Xdrive2mnt(df)
    df_AIs.append(df)




Metrics_vol = []
metrics_save_path = 'Metrics_%s_vs_Yen_CUIMCfirst.csv'%'ScaleNAS9SlicesAccLiver'
CTs = df_CTs["Image File Path"].values.tolist()

for ct in CTs:
    df_image = get_dicom_header_df( ct )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)


    df_ct_Yen = df_Yen[df_Yen["Image File Path"]==ct]
    df_ct_AI , dataset_id = get_onect_from_list(df_AIs , ct)
    # break
    if (df_ct_AI is None):
        if df_ct_Yen.shape[0]:
            fn = df_ct_Yen.shape[0]
            print('{} has {} FNs!'.format(ct , fn))   
        continue

    if not df_ct_Yen.shape[0]:
        fp = df_ct_AI.shape[0]
        print( '{} has {} FPs'.format(ct , fp)  )
        continue
    else:
        print(ct)

    raws = df_ct_AI["Contour File Path"].values.tolist()
    vol_pred = raws2mask(raws , D_z_index, mask_vol = None)
    connectivity = 2
    from skimage import measure
    labels_pred=measure.label(vol_pred,connectivity=connectivity)
    l_pred,c_pred = np.unique(labels_pred , return_counts=True)
    ix2 = l_pred>0
    l_pred = l_pred[ix2] #background pixels are labeled as 0, so we exclude them
    c_pred = c_pred[ix2]
    if len(l_pred)!= len(raws):
        print("warning! raws overlaped on {}".format(ct))


    for _ , row in df_ct_Yen.iterrows():

        Yen_raw = wr.read(row['Contour File Path'])
        gt_vol = initialize_mask_vol(Yen_raw , D_z_index)

        slice_list = Yen_raw.get_instance_number_array()
        for j, one in enumerate(slice_list):
            mask = Yen_raw.get_mask_image(j)
            gt_vol[D_z_index[one]] = mask
        
        hit = vols_seg_results(gt_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
        Metrics_vol.extend(hit)

        _n = len(Metrics_vol)
        if _n%100==0 or _n in [1,2,5,10,30,50]:
            df_metrics = pd.DataFrame(Metrics_vol, 
                                    columns = ['file_name','igt','merge','#gt','#pred',
                                                'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                                'area_gt','area_pred','intersection','union']) 
            df_metrics.to_csv(metrics_save_path)
        

/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0003/D2019_10_07/E2122/CT/S0002_2131
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0004/D2019_03_04/E9409/CT/S0016_5317
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0006/D2018_12_11/E8314/CT/S0002_5268
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0008/D2019_12_19/E2450/CT/S0002_7253
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0009/D2020_01_22/E0578/CT/S0018_3927
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0019/D2020_01_07/E8279/CT/S0005_2496
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0020/D2019_01_09/E2250/CT/S0005_1575
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0024/D2018_11_26/E0507/CT/S0006_5482
/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0026/D2019_04_12/E3925/CT/S0006_9366
/mnt/fast-disk1/mjc/AutoRecist

In [13]:
df_metrics = pd.DataFrame(Metrics_vol, 
                        columns = ['file_name','igt','merge','#gt','#pred',
                                    'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                    'area_gt','area_pred','intersection','union']) 
df_metrics.to_csv(metrics_save_path)

In [14]:
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,405.000,405.000,405.000,405.000,405.000,405.000,405.000,405.000,405.000,405.000,405.000,405.000
mean,1.032,0.000,1.000,9.595,0.330,0.423,2.475,0.517,7985.679,8910.817,5716.812,11179.684
std,0.214,0.000,0.000,7.219,0.288,0.344,20.207,0.392,35262.667,35950.585,26380.801,43633.240
min,1.000,0.000,1.000,1.000,0.000,0.000,0.000,0.000,8.000,0.000,0.000,8.000
5%,1.000,0.000,1.000,1.000,0.000,0.000,0.000,0.019,43.200,0.000,0.000,46.600
25%,1.000,0.000,1.000,4.000,0.000,0.000,0.000,0.136,219.000,0.000,0.000,285.000
50%,1.000,0.000,1.000,7.000,0.351,0.520,0.090,0.426,753.000,414.000,237.000,1002.000
75%,1.000,0.000,1.000,18.000,0.583,0.736,0.415,1.000,2525.000,2158.000,1126.000,3428.000
95%,1.000,0.000,1.000,23.000,0.773,0.872,2.989,1.000,27016.400,40593.200,16224.200,44590.800
max,4.000,0.000,1.000,23.000,0.940,0.969,354.027,1.000,446885.000,297966.000,275862.000,468989.000


In [15]:
df_metrics[df_metrics.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000,254.000
mean,1.035,0.000,1.000,10.146,0.522,0.666,0.442,0.284,12007.886,11781.311,9073.839,14715.358
std,0.185,0.000,0.000,7.333,0.182,0.169,0.623,0.244,44015.362,41235.168,32877.569,51626.750
min,1.000,0.000,1.000,1.000,0.147,0.256,0.000,0.000,29.000,24.000,17.000,89.000
5%,1.000,0.000,1.000,1.000,0.189,0.318,0.003,0.009,116.600,107.950,58.650,186.950
25%,1.000,0.000,1.000,4.000,0.397,0.568,0.074,0.079,351.500,342.500,205.250,511.500
50%,1.000,0.000,1.000,7.000,0.532,0.694,0.228,0.212,1014.500,936.000,719.500,1259.500
75%,1.000,0.000,1.000,19.000,0.665,0.799,0.496,0.431,4035.500,3354.250,2602.500,5153.000
95%,1.000,0.000,1.000,23.000,0.801,0.890,1.836,0.782,53398.350,43903.000,38525.050,56300.700
max,2.000,0.000,1.000,23.000,0.940,0.969,4.310,0.852,446885.000,297966.000,275862.000,468989.000


In [16]:
dfinnermerge = pd.merge(df_metrics,df_Yen,how='inner',left_on='file_name' , right_on='Contour File Path')
for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

file_name 405
igt 3
merge 1
#gt 1
#pred 16
iou_score 277
dice_score 277
over_seg 267
under_seg 267
area_gt 369
area_pred 247
intersection 264
union 381
Image File Path 72
Contour File Path 405
Raw File Name 405
Uni 396
Perp 396
Bi 405
Volume 405
In total, 72 CT series; 379 AI detections 
sensitivity is 0.681(276/405) FP-rate is 1.4 per CT-serie at threshold 0
sensitivity is 0.649(263/405) FP-rate is 1.6 per CT-serie at threshold 0.1
sensitivity is 0.640(259/405) FP-rate is 1.7 per CT-serie at threshold 0.2
sensitivity is 0.627(254/405) FP-rate is 1.7 per CT-serie at threshold 0.25
sensitivity is 0.516(209/405) FP-rate is 2.4 per CT-serie at threshold 0.5


In [17]:
#subgroup analysis for gt_lesion >=10mm

Uni_thresh = 10
df_subgroup = dfinnermerge[dfinnermerge['Uni']>=Uni_thresh]
df_subgroup.describe([.05, .25, .5, .75, .95])
df_subgroup[df_subgroup.dice_score>0.25].describe([.05, .25, .5, .75, .95])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union,Uni,Perp,Bi,Volume
count,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000,220.000
mean,1.036,0.000,1.000,10.659,0.540,0.682,0.350,0.290,13840.800,13566.295,10459.309,16947.786,31.335,21.695,1164.696,39080.241
std,0.188,0.000,0.000,7.362,0.181,0.166,0.460,0.242,47041.629,44050.011,35133.243,55151.706,27.754,18.339,2599.101,163340.504
min,1.000,0.000,1.000,1.000,0.148,0.258,0.000,0.000,125.000,44.000,41.000,180.000,10.034,4.642,50.501,265.378
5%,1.000,0.000,1.000,1.000,0.192,0.323,0.003,0.016,219.600,173.950,128.950,287.400,11.210,7.734,89.577,591.850
25%,1.000,0.000,1.000,5.000,0.430,0.601,0.066,0.085,602.000,512.500,363.750,730.000,15.545,11.535,171.999,1405.685
50%,1.000,0.000,1.000,8.000,0.559,0.717,0.212,0.231,1276.500,1224.500,924.500,1702.500,22.374,15.291,319.517,3167.850
75%,1.000,0.000,1.000,19.000,0.679,0.809,0.442,0.437,4781.250,4833.250,3395.250,6198.000,35.737,24.077,785.710,11969.425
95%,1.000,0.000,1.000,23.000,0.810,0.895,1.216,0.776,56808.150,53750.050,47235.700,57559.800,95.948,51.986,4708.741,161058.300
max,2.000,0.000,1.000,23.000,0.940,0.969,2.994,0.852,446885.000,297966.000,275862.000,468989.000,200.071,107.569,20459.460,1973380.000


In [18]:
def detection_performance(dfinnermerge):
    pts = dfinnermerge["Image File Path"].values.tolist()
    FPs = []
    for onept in list(set(pts)):
        df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
        assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
        fp = max(df_onept["#pred"])
        FPs.append(fp)
    print("="*80)
    print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

    dices = dfinnermerge.dice_score.tolist()

    for th in [0, 0.1, 0.2, 0.25, 0.5]:
        TP = [p>th for p in dices ]
        assert( len(TP) == len(dices))
        fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
        print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

In [19]:
detection_performance(df_subgroup)

In total, 66 CT series; 369 AI detections 
sensitivity is 0.751(241/321) FP-rate is 1.9 per CT-serie at threshold 0
sensitivity is 0.710(228/321) FP-rate is 2.1 per CT-serie at threshold 0.1
sensitivity is 0.698(224/321) FP-rate is 2.2 per CT-serie at threshold 0.2
sensitivity is 0.685(220/321) FP-rate is 2.3 per CT-serie at threshold 0.25
sensitivity is 0.579(186/321) FP-rate is 2.8 per CT-serie at threshold 0.5
